### 1) 필요한 라이브러리 설치

In [1]:
import pandas as pd
import os
import time

import torch
from tqdm import tqdm
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.
from transformers import AutoModelForCausalLM, AutoTokenizer


In [ ]:
config_data = {
    "PATH": {"DATA_PATH" : "../data/",
        "RESULT_PATH" : "../prediction/"},
    "general": {"data_path": "../data/", # 모델 생성에 필요한 데이터 경로를 사용자 환경에 맞게 지정합니다.
        "model_name": "Qwen3:8b", #"digit82/kobart-summarization", # 불러올 모델의 이름을 사용자 환경에 맞게 지정할 수 있습니다.
        "output_dir": "../prdiction" # 모델의 최종 출력 값을 저장할 경로를 설정합니다.
    },
    "tokenizer": {
        "max_len": 512,
        # 특정 단어들이 분해되어 tokenization이 수행되지 않도록 special_tokens을 지정해줍니다.
    },
CFS = { # 데이터 경로를 지정해줍니다.
DATA_PATH = "../data/"
RESULT_PATH = "./prediction/"

### 2) qwen3 model from transfomers

In [3]:
model_name = "Qwen/Qwen3-8B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

# prepare the model input
# prompt = "Give me a short introduction to large language model."
# messages = [
#     {"role": "user", "content": prompt}
# # ]
# text = tokenizer.apply_chat_template(
#     messages,
#     tokenize=False,
#     add_generation_prompt=True,
#     enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
# )
# model_inputs = tokenizer([text], return_tensors="pt").to(model.device)



Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

### 4) 데이터 불러오기
- 실험에서 쓰일 데이터를 load합니다.

In [3]:
# 데이터 경로를 지정해줍니다.
DATA_PATH = "../data/"
RESULT_PATH = "./prediction/"

# train data의 구조와 내용을 확인합니다.
train_df = pd.read_csv(os.path.join(DATA_PATH,'train.csv'))
train_df.tail()

,fname,dialogue,summary,topic
12452,train_12455,#Person1#: 안녕하세요. 혹시 맨체스터에서 오신 Mr. Green 맞으신가요...,Tan Ling은 흰머리와 수염이 특징인 Mr. Green을 맞이하여 호텔로 안내합...,호텔 안내
12453,train_12456,"#Person1#: Mister Ewing이 우리 회의장에 4시에 오라고 했지, 맞...",#Person1#과 #Person2#는 Mister Ewing의 요청에 따라 회의장...,회의 준비
12454,train_12457,#Person1#: 오늘 어떻게 도와드릴까요?\n#Person2#: 차를 빌리고 싶...,#Person2#는 #Person1#의 도움으로 5일 동안 소형차를 대여합니다.,차량 대여
12455,train_12458,#Person1#: 너 오늘 좀 기분 안 좋아 보인다? 무슨 일 있어?\n#Pers...,#Person2#의 어머니가 직장을 잃으셨다. #Person2#는 어머니가 우울해하...,실직과 대처
12456,train_12459,"#Person1#: 엄마, 나 다음 주 토요일에 이모부네 가족 보러 가는데, 오늘 ...",#Person1#은 다음 주 토요일에 이모부네 가족을 방문하기 위해 짐을 싸야 하는...,가족 방문 준비


In [4]:
# 모델 성능에 대한 평가 지표를 정의합니다. 본 대회에서는 ROUGE 점수를 통해 모델의 성능을 평가합니다.
rouge = Rouge()
def compute_metrics(pred, gold):
    results = rouge.get_scores(pred, gold, avg=True)
    result = {key: value["f"] for key, value in results.items()}
    return result

In [16]:
# Dialogue를 입력으로 받아, Solar Chat API에 보낼 Prompt를 생성하는 함수를 정의합니다.
def build_prompt(dialogue):
    system_prompt = "You are an expert in the field of dialogue summarization. " \
    "If a person's name appears, please use the actual name instead of Person1 or Person2." \
    "Please summarize the following dialogue to only a sentnce in Korean." \
    

    user_prompt = f"Dialogue:\n{dialogue}\n\nSummary:\n"
    
    prompt = system_prompt + "\n\n" + user_prompt
    return prompt

In [30]:
def summarization(dialogue):
    
    prompt = build_prompt(dialogue)  
    inputs = tokenizer(prompt, return_tensors="pt",).to(model.device)
    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.3,
            top_p=0.9,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            early_stopping=True,
        )
        output_ids = generated_ids[0][len(inputs.input_ids[0]):].tolist()

        try:
            index = len(output_ids) - output_ids[::-1].index(tokenizer.convert_tokens_to_ids("</think>"))  # THINK_TOKEN_ID는 "</think>" 토큰 ID
        except ValueError:
            index = 0

        # 사고 부분 제거
        filtered_output_ids = output_ids[index:]
        content = tokenizer.decode(filtered_output_ids, skip_special_tokens=True)

        # output_ids = generated_ids[0][len(inputs.input_ids[0]):].tolist() 
        # # parsing thinking content
        # try:
        #     # rindex finding 151668 (</think>)
        #     index = len(output_ids) - output_ids[::-1].index(151668)
        # except ValueError:
        #     index = 0

        # thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
        # content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

        # print("thinking content:", thinking_content)
        # print("content:", content)
    # # 출력 추출(프롬프트 부분 제외)
    # generated = output[:, inputs.input_ids.shape[1]:]
    # summary = tokenizer.decode(generated[0], skip_special_tokens=True)
    
    return content


In [31]:
# Train data 중 처음 3개의 대화를 요약합니다.
def test_on_train_data(num_samples=3):
    for idx, row in train_df[:num_samples].iterrows():
        dialogue = row['dialogue']
        summary = summarization(dialogue)
        print(summary)
        print(f"Dialogue:\n{dialogue}\n")
        print(f"Pred Summary: {summary}\n")
        print(f"Gold Summary: {row['summary']}\n")
        print("=="*50)

In [2]:

model_name = "Qwen/Qwen3-8B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

# prepare the model input
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

thinking content: <think>
Okay, the user wants a short introduction to large language models. Let me start by defining what they are. They're AI systems trained on massive amounts of text data, right? So I should mention that they're based on deep learning and natural language processing.

Next, I need to highlight their key features. They can understand and generate human-like text, which is important. Maybe mention their ability to handle multiple languages and tasks like answering questions, writing stories, coding, etc. Also, their scale is crucial—billions of parameters. That's a key point.

Wait, should I explain parameters? Maybe not in detail, just mention it as part of their complexity. Also, applications are important. Examples like chatbots, content creation, data analysis. But keep it concise.

I should also touch on their training process. They're trained on diverse text data to learn patterns and relationships. That helps them generate coherent responses. Maybe mention th

In [32]:
if __name__ == "__main__":
    test_on_train_data()

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Dr. Hawkins은 Mr. Smith에게 건강검진의 중요성을 강조하고, 담배를 끊는 데 도움을 줄 수 있는 방법을 제안합니다.
Okay, let's tackle this. The user wants a one-sentence summary in Korean of the dialogue between Dr. Hawkins and Mr. Smith. First, I need to identify the main points. 

Dr. Hawkins starts by mentioning that Mr. Smith hasn't had a check-up in five years and advises annual check-ups. Mr. Smith says he doesn't need one if he's not sick. Dr. Hawkins argues that early detection is key and urges him to come yearly. Then, during the exam, Dr. Hawkins notices Mr. Smith smokes and warns about the risks of smoking, suggesting quitting and offering help like classes and medication. Mr. Smith admits he's tried quitting many times but can't break the habit. Dr. Hawkins offers more information before he leaves.

So the key elements are: emphasizing the importance of annual check-ups, warning about smoking risks, and offering assistance to quit. Need to condense this into one sentence in Korean, using the actual names. Let me ch

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Okay, let's tackle this dialogue summary. The user wants a single-sentence summary in Korean. First, I need to understand the conversation between Person1 (Dr. Peters) and Person2 (Mrs. Parker). 

Dr. Peters starts by greeting Mrs. Parker and asks how she's doing. Mrs. Parker responds and mentions she and Ricky came for vaccinations. Dr. Peters checks Ricky's vaccination records and notes he's received polio, tetanus, and hepatitis B. He mentions Ricky is 14 months old and needs hepatitis A, chickenpox, and measles vaccines. Mrs. Parker asks about mumps and rubella, and Dr. Peters says they can do the rest in a few weeks. Mrs. Parker also mentions she needs a tetanus booster since her last one was 15 years ago. Dr. Peters agrees to check the records and prepare for the booster. He then tells Mrs. Parker to hold Ricky's arm for the shot.

Now, the key points are: Ricky's vaccination schedule, the need for additional vaccines, Mrs. Parker's tetanus booster, and the preparation for the sh

Validation Dataset을 이용하여 요약을 진행하고, 성능을 평가해 봅니다.

In [15]:
# Validation data의 대화를 요약하고, 점수를 측정합니다.
def validate(num_samples=-1):
    val_samples = val_df[:num_samples] if num_samples > 0 else val_df
    
    scores = []
    for idx, row in tqdm(val_samples.iterrows(), total=len(val_samples)):
        dialogue = row['dialogue']
        summary = summarization(dialogue)
        results = compute_metrics(summary, row['summary'])
        avg_score = sum(results.values()) / len(results)
        
        scores.append(avg_score)
        
    val_avg_score = sum(scores) / len(scores)

    print(f"Validation Average Score: {val_avg_score}")

In [13]:
if __name__ == "__main__":
    validate(100) # 100개의 validation sample에 대한 요약을 수행합니다.
    
    # 전체 validation data에 대한 요약을 수행하고 싶은 경우 아래와 같이 실행합니다.
    # validate() 

100%|██████████| 100/100 [02:14<00:00,  1.34s/it]

Validation Average Score: 0.09606867456611186


## 2. Solar Chat API로 요약하기
- Solar Chat API을 이용하여 test dataset에 포함된 dialogue를 요약하고 제출용 파일을 생성합니다.

In [16]:
def inference():
    test_df = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))

    summary = []
    start_time = time.time()
    for idx, row in tqdm(test_df.iterrows(), total=len(test_df)):
        dialogue = row['dialogue']
        summary.append(summarization(dialogue))
        
        # Rate limit 방지를 위해 1분 동안 최대 100개의 요청을 보내도록 합니다.
        if (idx + 1) % 100 == 0:
            end_time = time.time()
            elapsed_time = end_time - start_time
            
            if elapsed_time < 60:
                wait_time = 60 - elapsed_time + 5
                print(f"Elapsed time: {elapsed_time:.2f} sec")
                print(f"Waiting for {wait_time} sec")
                time.sleep(wait_time)
            
            start_time = time.time()
    
    output = pd.DataFrame(
        {
            "fname": test_df['fname'],
            "summary" : summary,
        }
    )
    
    if not os.path.exists(RESULT_PATH):
        os.makedirs(RESULT_PATH)
    output.to_csv(os.path.join(RESULT_PATH, "output_solar.csv"), index=False)

    return output

In [17]:
if __name__ == "__main__":
    output = inference()

  0%|          | 0/499 [00:00<?, ?it/s]

100%|██████████| 499/499 [05:35<00:00,  1.49it/s]


In [29]:
output  # 각 대화문에 대한 요약문이 출력됨을 확인할 수 있습니다.

,fname,summary
0,test_0,모든 직원에게 즉시 메시지 프로그램 사용을 금지하고 이메일과 공식 메모로만 통신을 ...
1,test_1,"#Person2#는 교통체증으로 인해 늦게 도착했고, #Person1#은 출퇴근 시..."
2,test_2,"마샤와 히어로가 두 달간 별거 끝에 이혼을 신청했으며, 마샤가 양육권을 가지기로 했다."
3,test_3,"#Person1#: 정말 멋진 파티야. #Person2#: 응, 너는 항상 모두에게..."
4,test_4,"올림픽 스타디움은 6월에 완공되며, 5000석의 좌석과 육상 트랙, 점프 구간이 있..."
...,...,...
494,test_495,잭은 찰리에게 학교가 끝난 후 집에 와서 새로 산 캐릭터 커스터마이징이 가능한 비디...
495,test_496,#Person2#는 아내와 함께 다양한 레코드를 구입하던 중 시골 음악 레코드에 더...
496,test_497,"Person1은 Person2에게 세탁기 사용법을 물어보고, 비누를 너무 많이 사용..."
497,test_498,Steve가 1년만에 만난 Matthew에게 자신의 이웃이었던 Mrs. Thou의 ...


In [18]:
output  # 각 대화문에 대한 요약문이 출력됨을 확인할 수 있습니다.

,fname,summary
0,test_0,모든 직원에게 즉시 메시지 프로그램 사용을 금지하고 이메일과 공식 메모로만 통신을 ...
1,test_1,"#Person2#는 교통체증으로 인해 늦게 도착했고, #Person1#은 출퇴근 시..."
2,test_2,"마샤와 히어로가 두 달간 별거 끝에 이혼을 신청했으며, 마샤가 양육권을 가지기로 했다."
3,test_3,"#Person1#: 정말 멋진 파티야. #Person2#: 응, 너는 항상 모두에게..."
4,test_4,"올림픽 스타디움은 6월에 완공되며, 5000석의 좌석과 육상 트랙, 점프 구간이 있..."
...,...,...
494,test_495,잭은 찰리에게 학교가 끝난 후 집에 와서 새로 산 캐릭터 커스터마이징이 가능한 비디...
495,test_496,#Person2#는 아내와 함께 다양한 레코드를 구입하던 중 시골 음악 레코드에 더...
496,test_497,"Person1은 Person2에게 세탁기 사용법을 물어보고, 비누를 너무 많이 사용..."
497,test_498,Steve가 1년만에 만난 Matthew에게 자신의 이웃이었던 Mrs. Thou의 ...


## 3. Prompt Engineering
- Prompt engineering을 통해 요약 성능 향상을 시도합니다.

In [ ]:
# Few-shot prompt를 생성하기 위해, train data의 일부를 사용합니다.
few_shot_samples = train_df.sample(1)

sample_dialogue1 = few_shot_samples.iloc[0]['dialogue']
sample_summary1 = few_shot_samples.iloc[0]['summary']

print(f"Sample Dialogue1:\n{sample_dialogue1}\n")
print(f"Sample Summary1: {sample_summary1}\n")

In [ ]:
# Prompt를 생성하는 함수를 수정합니다.
def build_prompt(dialogue):
    system_prompt = "You are a expert in the field of dialogue summarization, summarize the given dialogue in a concise manner. Follow the user's instruction carefully and provide a summary that is relevant to the dialogue."

    user_prompt = (
        "Following the instructions below, summarize the given document.\n"
        "Instructions:\n"
        "1. Read the provided sample dialogue and corresponding summary.\n"
        "2. Read the dialogue carefully.\n"
        "3. Following the sample's style of summary, provide a concise summary of the given dialogue.\n\n"
        "Sample Dialogue:\n"
        f"{sample_dialogue1}\n\n"
        "Sample Summary:\n"
        f"{sample_summary1}\n\n"
        "Dialogue:\n"
        f"{dialogue}\n\n"
        "Summary:\n"
    )
    
    return [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_prompt
        }
    ]

In [ ]:
# 변경된 prompt를 사용하여, train data 중 처음 3개의 대화를 요약하고, 결과를 확인합니다.
if __name__ == "__main__":
    test_on_train_data()

In [ ]:
# 변경된 prompt를 사용하여, validation data의 대화를 요약하고, 점수를 측정합니다.
if __name__ == "__main__":
    validate(100)

다른 방식으로 Few-shot sample을 제공하여 Prompt를 구성해 봅니다.

In [ ]:
# Few-shot sample을 다른 방식으로 사용하여 prompt를 생성합니다.
def build_prompt(dialogue):
    system_prompt = "You are a expert in the field of dialogue summarization, summarize the given dialogue in a concise manner. Follow the user's instruction carefully and provide a summary that is relevant to the dialogue."

    few_shot_user_prompt_1 = (
        "Following the instructions below, summarize the given document.\n"
        "Instructions:\n"
        "1. Read the provided sample dialogue and corresponding summary.\n"
        "2. Read the dialogue carefully.\n"
        "3. Following the sample's style of summary, provide a concise summary of the given dialogue. Be sure that the summary is simple but captures the essence of the dialogue.\n\n"
        "Dialogue:\n"
        f"{sample_dialogue1}\n\n"
        "Summary:\n"
    )
    
    few_shot_assistant_prompt_1 = sample_summary1
    
    user_prompt = (
        "Dialogue:\n"
        f"{dialogue}\n\n"
        "Summary:\n"
    )
    
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": few_shot_user_prompt_1},
        {"role": "assistant", "content": few_shot_assistant_prompt_1},
        {"role": "user", "content": user_prompt},
    ]

In [ ]:
# 변경된 prompt를 사용하여, train data 중 처음 3개의 대화를 요약하고, 결과를 확인합니다.
if __name__ == "__main__":
    test_on_train_data()

In [ ]:
# 변경된 prompt를 사용하여, validation data의 대화를 요약하고, 점수를 측정합니다.
if __name__ == "__main__":
    validate(100)

### (선택) 변경된 Prompt로 test dataset에 대한 요약을 진행합니다.
- 변경된 prompt를 통해 점수가 개선되었다면, test dataset에 대한 요약을 진행하고 제출합니다.

In [ ]:
# 변경된 prompt를 사용하여, test data의 대화를 요약하고, 결과를 확인합니다.
if __name__ == "__main__":
    output = inference()

In [ ]:
output